In [1]:
import os
import cv2
import face_recognition
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from imutils.video import VideoStream
from eye_status import * 


In [2]:
def init():
    face_cascPath = 'haarcascade_frontalface_alt.xml'
    # face_cascPath = 'lbpcascade_frontalface.xml'

    open_eye_cascPath = 'haarcascade_eye_tree_eyeglasses.xml'
    left_eye_cascPath = 'haarcascade_lefteye_2splits.xml'
    right_eye_cascPath ='haarcascade_righteye_2splits.xml'
    dataset = 'faces'

    face_detector = cv2.CascadeClassifier(face_cascPath)
    open_eyes_detector = cv2.CascadeClassifier(open_eye_cascPath)
    left_eye_detector = cv2.CascadeClassifier(left_eye_cascPath)
    right_eye_detector = cv2.CascadeClassifier(right_eye_cascPath)

    print("[LOG] Opening webcam ...")
    video_capture = VideoStream(src=0).start()
    #video_capture = cv2.VideoCapture(0)

    model = load_model()


    print("[LOG] Collecting images ...")
    images = []
    for direc, _, files in tqdm(os.walk(dataset)):
        for file in files:
            if file.endswith("jpg"):
                images.append(os.path.join(direc,file))
    return (model,face_detector, open_eyes_detector, left_eye_detector,right_eye_detector, video_capture, images) 

In [3]:
def process_and_encode(images):
    # initialize the list of known encodings and known names
    known_encodings = []
    known_names = []
    print("[LOG] Encoding faces ...")

    for image_path in tqdm(images):
        # Load image
        image = cv2.imread(image_path)
        # Convert it from BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
     
        # detect face in the image and get its location (square boxes coordinates)
        boxes = face_recognition.face_locations(image, model='hog')

        # Encode the face into a 128-d embeddings vector
        encoding = face_recognition.face_encodings(image, boxes)

        # the person's name is the name of the folder where the image comes from
        name = image_path.split(os.path.sep)[-2]

        if len(encoding) > 0 : 
            known_encodings.append(encoding[0])
            known_names.append(name)

    return {"encodings": known_encodings, "names": known_names}

def isBlinking(history, maxFrames):
    """ @history: A string containing the history of eyes status 
         where a '1' means that the eyes were closed and '0' open.
        @maxFrames: The maximal number of successive frames where an eye is closed """
    for i in range(maxFrames):
        pattern = '1' + '0'*(i+1) + '1'
        if pattern in history:
            return True
    return False

In [4]:
def detect_and_display(model, video_capture, face_detector, open_eyes_detector, left_eye_detector, right_eye_detector, data, eyes_detected):
        frame = video_capture.read()
        # resize the frame
        frame = cv2.resize(frame, (0, 0), fx=0.6, fy=0.6)

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Detect faces
        faces = face_detector.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(50, 50),
            flags=cv2.CASCADE_SCALE_IMAGE
        )

        # for each detected face
        for (x,y,w,h) in faces:
            # Encode the face into a 128-d embeddings vector
            encoding = face_recognition.face_encodings(rgb, [(y, x+w, y+h, x)])[0]

            # Compare the vector with all known faces encodings
            matches = face_recognition.compare_faces(data["encodings"], encoding)

            # For now we don't know the person name
            name = "Unknown"

            # If there is at least one match:
            if True in matches:
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1

                # determine the recognized face with the largest number of votes
                name = max(counts, key=counts.get)

            face = frame[y:y+h,x:x+w]
            gray_face = gray[y:y+h,x:x+w]

            eyes = []
            
            # Eyes detection
            # check first if eyes are open (with glasses taking into account)
            open_eyes_glasses = open_eyes_detector.detectMultiScale(
                gray_face,
                scaleFactor=1.1,
                minNeighbors=5,
                minSize=(30, 30),
                flags = cv2.CASCADE_SCALE_IMAGE
            )
            # if open_eyes_glasses detect eyes then they are open 
            if len(open_eyes_glasses) == 2:
                eyes_detected[name]+='1'
                for (ex,ey,ew,eh) in open_eyes_glasses:
                    cv2.rectangle(face,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
            
            # otherwise try detecting eyes using left and right_eye_detector
            # which can detect open and closed eyes                
            else:
                # separate the face into left and right sides
                left_face = frame[y:y+h, x+int(w/2):x+w]
                left_face_gray = gray[y:y+h, x+int(w/2):x+w]

                right_face = frame[y:y+h, x:x+int(w/2)]
                right_face_gray = gray[y:y+h, x:x+int(w/2)]

                # Detect the left eye
                left_eye = left_eye_detector.detectMultiScale(
                    left_face_gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )

                # Detect the right eye
                right_eye = right_eye_detector.detectMultiScale(
                    right_face_gray,
                    scaleFactor=1.1,
                    minNeighbors=5,
                    minSize=(30, 30),
                    flags = cv2.CASCADE_SCALE_IMAGE
                )

                eye_status = '1' # we suppose the eyes are open

                # For each eye check wether the eye is closed.
                # If one is closed we conclude the eyes are closed
                for (ex,ey,ew,eh) in right_eye:
                    color = (0,255,0)
                    pred = predict(right_face[ey:ey+eh,ex:ex+ew],model)
                    if pred == 'closed':
                        eye_status='0'
                        color = (0,0,255)
                    cv2.rectangle(right_face,(ex,ey),(ex+ew,ey+eh),color,2)
                for (ex,ey,ew,eh) in left_eye:
                    color = (0,255,0)
                    pred = predict(left_face[ey:ey+eh,ex:ex+ew],model)
                    if pred == 'closed':
                        eye_status='0'
                        color = (0,0,255)
                    cv2.rectangle(left_face,(ex,ey),(ex+ew,ey+eh),color,2)
                eyes_detected[name] += eye_status

            # Each time, we check if the person has blinked
            # If yes, we display its name
            if isBlinking(eyes_detected[name],3):
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                # Display name
                y = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(frame, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 255, 0), 2)

        return frame

In [5]:
if __name__ == "__main__":
    (model, face_detector, open_eyes_detector,left_eye_detector,right_eye_detector, video_capture, images) = init()
    data = process_and_encode(images)

    eyes_detected = defaultdict(str)
    while True:
        frame = detect_and_display(model, video_capture, face_detector, open_eyes_detector,left_eye_detector,right_eye_detector, data, eyes_detected)
        cv2.imshow("Face Liveness Detector", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()
    video_capture.stop()

[LOG] Opening webcam ...


0it [00:00, ?it/s]
0it [00:00, ?it/s]

[LOG] Collecting images ...
[LOG] Encoding faces ...


[[208 208 207 ... 215 215 213]
 [209 209 208 ... 216 216 216]
 [209 209 209 ... 222 221 219]
 ...
 [200 200 199 ... 195 196 199]
 [199 200 200 ... 196 198 201]
 [199 199 200 ... 198 201 203]]
[[208 207 207 208 207 208 210 212 215 215 216 217 216 216 219 222 222 222
  222 222 222 219 215 214]
 [209 209 209 208 207 207 208 211 213 214 215 217 217 217 218 218 222 223
  222 224 225 224 221 219]
 [210 210 209 206 204 201 202 206 208 209 210 211 211 210 209 211 215 220
  221 220 222 225 224 223]
 [208 210 206 200 197 198 197 195 198 199 200 196 191 187 188 192 200 208
  210 213 218 222 223 224]
 [207 205 197 188 186 186 184 186 190 195 188 169 154 150 157 166 181 192
  199 202 208 215 220 223]
 [200 193 181 176 179 184 181 182 191 179 156 147 143 139 142 148 162 174
  184 194 197 205 211 217]
 [189 182 180 177 182 184 178 177 164 151 144 146 151 158 162 170 171 167
  169 183 192 193 203 212]
 [185 183 189 194 197 195 187 181 182 190 201 205 206 205 204 203 201 196
  192 192 195 197 203 209]


[[205 205 204 ... 194 194 195]
 [207 206 205 ... 197 195 196]
 [207 206 205 ... 200 198 197]
 ...
 [199 198 197 ... 196 197 195]
 [200 199 197 ... 198 198 196]
 [201 199 197 ... 199 198 197]]
[[205 204 202 199 196 203 206 207 208 207 209 208 209 208 205 204 206 204
  204 203 199 195 195 195]
 [207 205 206 206 205 207 209 211 210 210 212 213 212 210 210 210 212 211
  208 208 204 201 200 197]
 [207 206 206 206 208 209 211 212 211 212 214 214 214 214 215 216 219 218
  215 211 208 206 204 202]
 [207 207 208 208 207 209 209 213 210 213 215 214 214 217 220 220 221 219
  218 215 213 213 207 204]
 [207 206 207 207 207 208 208 212 213 216 217 216 215 217 220 222 220 220
  220 219 219 215 214 209]
 [209 207 207 206 203 204 205 209 210 213 213 215 215 216 217 220 221 221
  222 222 223 221 221 215]
 [209 208 202 199 195 197 200 203 206 205 208 208 208 207 209 214 219 218
  218 221 224 223 224 222]
 [206 201 192 187 190 188 189 193 194 196 188 180 178 180 185 196 205 208
  211 216 219 222 225 227]


[[206 206 206 ... 203 200 197]
 [205 205 204 ... 206 202 199]
 [205 204 204 ... 208 204 202]
 ...
 [201 201 201 ... 193 192 194]
 [200 200 200 ... 193 193 195]
 [199 198 198 ... 193 193 195]]
[[206 205 204 203 202 203 204 207 209 210 209 209 210 211 209 209 209 207
  208 210 208 206 203 198]
 [205 204 203 202 203 204 206 207 208 210 209 210 210 211 210 211 211 211
  214 214 214 210 207 202]
 [205 205 205 204 205 205 206 206 207 209 209 210 212 213 211 211 214 215
  215 215 214 212 211 209]
 [207 207 206 205 207 205 205 206 205 207 209 212 214 214 212 213 215 217
  216 216 218 218 217 212]
 [206 207 207 207 204 202 198 199 202 205 208 207 208 211 211 212 213 215
  218 218 218 219 220 217]
 [207 207 207 204 198 191 191 193 193 197 198 199 199 200 201 198 200 206
  212 216 215 218 221 220]
 [206 204 200 192 184 180 178 180 180 186 190 191 181 168 166 168 175 189
  198 205 204 211 217 219]
 [196 195 182 173 169 171 179 176 174 187 184 164 151 142 135 139 148 165
  177 191 194 200 210 214]


[[205 205 204 ... 190 189 190]
 [205 205 204 ... 193 192 192]
 [205 205 205 ... 194 194 193]
 ...
 [195 194 195 ... 181 180 184]
 [194 191 187 ... 180 181 186]
 [192 184 178 ... 182 184 188]]
[[205 204 202 202 202 201 202 202 202 201 202 200 200 201 200 200 200 200
  199 196 194 192 190 190]
 [205 205 203 202 202 202 202 203 201 202 202 200 201 203 204 204 203 203
  202 200 199 197 194 193]
 [205 206 203 201 199 200 201 203 202 203 204 202 203 205 206 206 206 207
  207 205 202 201 196 195]
 [205 204 202 201 199 198 198 201 202 202 204 203 205 206 207 210 208 209
  210 209 206 205 202 200]
 [205 204 201 196 190 191 194 198 197 198 198 200 198 200 204 208 210 210
  210 211 211 211 210 207]
 [205 202 194 183 181 179 179 185 187 184 175 172 175 176 190 200 202 205
  209 212 213 214 214 214]
 [200 189 176 166 163 164 168 178 175 157 144 137 141 151 167 184 192 194
  201 209 212 213 215 217]
 [184 168 160 161 167 164 169 170 145 141 138 135 137 137 145 158 173 187
  192 199 207 209 212 215]
